In [1]:
from llama_index.vector_stores.pinecone import PineconeVectorStore
from pinecone import Pinecone
from dotenv import load_dotenv
import os

load_dotenv()


/home/isaiaszc/pathway/pathway-chatbot/backend/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:
pc = Pinecone()
index_name = os.getenv("PINECONE_INDEX_NAME")
index = pc.Index(index_name)
index.describe_index_stats()
vector_store = PineconeVectorStore(
    pinecone_index=index,
)


In [3]:
import pandas as pd

qa_df = pd.read_csv("qa_current.csv")
qa_df.head()


,compare?,better prompt?,Question,Ideal Answer,Generated Answer,Retrieved Content,Alirio score,Bam score,Dallan score,Isaias score,Rick score,Avg,Notes
0,X,X,How many credits for a Bachelors degree?,"Through BYU-Pathway Worldwide, BYU-Idaho and E...",All current bachelor's degrees at BYU-Idaho re...,Requirements / ( Online) Ensign Information NO...,1,1,1,1,1.0,1.0,We have information that still says that a bac...
1,X,NaN,How do I find the weekly devotional?,Weekly devotionals are available to missionari...,To find the weekly devotional for EnglishConne...,Help Them Download the Gospel Library App / Ar...,1,1,1,1,1.0,1.0,The Answer was directly addressed only to EC3....
2,X,X,Can I access my Zoom class from a link within ...,No. Students access their Zoom Gathering from...,"Yes, you can access your Zoom class from a lin...",How Students Access Your Zoom Room / Steps to ...,1,1,1,1,1.0,1.0,Uses external knowledge
3,NaN,X,How do I report gathering attendance and to whom?,Missionaries are not required to report Gather...,"To report gathering attendance, service missio...",How do I use Zoom to take attendance? / Introd...,1,1,2,1,1.0,1.2,Insufficient content
4,NaN,NaN,Should missionaries send the gathering zoom li...,Students should be directed to join their gath...,"Yes, it is recommended for missionaries to sen...",Open Your Zoom Room / WHO IS THIS ARTICLE FOR?...,1,2,1,1,1.0,1.2,Incorrect answer based on incorrect data/conte...


In [4]:
from app.engine import get_chat_engine


In [5]:
from llama_index.core.settings import Settings
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model_name = "text-embedding-3-large"

Settings.embed_model = OpenAIEmbedding(
    model=embed_model_name,
    embed_batch_size=100,
    max_retries=25,
    timeout=180,
    reuse_client=True,
    dimensions=3072,
)


In [6]:
Settings.llm.model


'gpt-3.5-turbo'

In [7]:
from llama_index.llms.openai import OpenAI

max_tokens = os.getenv("LLM_MAX_TOKENS")

Settings.llm = OpenAI(
    model="gpt-4o-mini",
    temperature=float(0),
    max_tokens=int(max_tokens) if max_tokens is not None else None,
)


In [8]:
Settings.llm.model


'gpt-4o-mini'

In [10]:
from test_tools import evaluate_response

messages = ""
chat_engine = get_chat_engine()
new_data = []
for index, row in qa_df.iterrows():
    question = row["Question"]
    ideal_answer = row["Ideal Answer"]
    chat_engine.reset()
    response = await chat_engine.achat(question)
    retrieved = "\n\n".join(
        [
            f"node_id: {idx+1}\n{node.metadata['url']}\n{node.text}"
            for idx, node in enumerate(response.source_nodes)
        ]
    )
    ia_evaluation = evaluate_response(
        question=question,
        ideal=ideal_answer,
        generated=response.response,
        nodes=retrieved,
    )
    ia_score, ia_explanation = ia_evaluation.split(" - ")
    # get only the number inside ()
    # ia_score = ia_score.split("(")[1].split(")")[0]

    updated_row = row.to_dict()
    updated_row.update(
        {
            "Question": question,
            "Ideal Answer": ideal_answer,
            "Query Engine Answer": response.response,
            "Retrieved Content": retrieved,
            "ia_score": ia_score,
            "ia_explanation": ia_explanation,
        }
    )
    new_data.append(updated_row)


Condensed question: How many credits for a Bachelors degree?
Context: We have 9 nodes:
node_id: 1
text: Program Strategies / Bachelor's Degree (90–96 credit hours) Our three-year bachelor's degrees are made up of three certificates, plus general courses. These degrees may help you progress within your career and earn additional income. In addition, courses taken as part of certificates and an associate degree count toward bachelor's degrees. View Bachelor's Degrees

node_id: 2
text: Web Development / Questions / What is the difference between a certificate, an associate degree, and a bachelor's degree? * A certificate is 12–15 credits and focuses on one area of study. * An associate degree is 60 credits and includes general studies. * A bachelor’s degree is 90 credits and includes a major field of study as well as general studies.

node_id: 3
text: Earn a bachelor's degree in three years through / Fewer credits. 100% online. Through , BYU-Idaho and Ensign College offer 90–96 credit deg

In [11]:
result_df = pd.DataFrame(new_data)
result_df.head()
result_df = result_df.dropna(subset=["Ideal Answer"])
result_df.to_csv("chatbot_answer.csv")


In [12]:
result_df


,compare?,better prompt?,Question,Ideal Answer,Generated Answer,Retrieved Content,Alirio score,Bam score,Dallan score,Isaias score,Rick score,Avg,Notes,Query Engine Answer,ia_score,ia_explanation
0,X,X,How many credits for a Bachelors degree?,"Through BYU-Pathway Worldwide, BYU-Idaho and E...",All current bachelor's degrees at BYU-Idaho re...,node_id: 1\nhttps://www.byupathway.edu/degrees...,1,1,1,1,1.0,1.0,We have information that still says that a bac...,A bachelor's degree at BYU-Idaho and Ensign Co...,5,The generated answer contains all the key fact...
1,X,NaN,How do I find the weekly devotional?,Weekly devotionals are available to missionari...,To find the weekly devotional for EnglishConne...,node_id: 1\nhttps://missionaries.prod.byu-path...,1,1,1,1,1.0,1.0,The Answer was directly addressed only to EC3....,"Sorry, I'm not able to answer this question. C...",3,The generated answer provides no substantive c...
